In [1]:
import pandas as pd 
import json
from collections import namedtuple
from os.path import exists

In [2]:
df = pd.read_csv('steam/steam_data.csv')
df

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...
...,...,...,...,...,...,...,...,...,...,...,...,...
81043,https://store.steampowered.com/bundle/2961/Ste...,-,-,-,-,-,-,-,-,Buy Steam: Rails to Riches - All Map Expansion...,-,-
81044,https://store.steampowered.com/bundle/3123/Det...,-,-,-,-,-,-,-,-,Buy Detective Hank and the Golden Sneeze - Gam...,-,-
81045,https://store.steampowered.com/bundle/3175/Fea...,-,-,-,-,-,-,-,-,-,-,-
81046,https://store.steampowered.com/bundle/3176/Fea...,-,-,-,-,-,-,-,-,-,-,-


Parse URLS to uuid

In [3]:
import re
url_list = []

def findid(row):
    result = re.search(string=row, pattern=r'\b/[0-9]*/')
    if result is not None:
        result2 = re.sub(string=result.group(0), pattern=r'/*', repl='')
        if result2 is not None:
            url_list.append(result2)

df['url'].apply(lambda x: findid(x))
steam_ids = pd.DataFrame(url_list)

In [4]:
steam_ids.head()

,0
0,945360
1,730
2,1097150
3,1158310
4,1085660


In [5]:
from steam import Steam
import requests
import json


file = open('./env.txt')
STEAM_KEY = file.read() 
data = []
steam = Steam(STEAM_KEY)
uuid='945360'

def getgamereviews(uuid): 
    result = requests.get('https://store.steampowered.com/appreviews/'+uuid+'?json=1cursor=20')
    return result.text

data.append(getgamereviews(uuid='945360'))
with open('test.json', 'w', encoding="utf-8") as f:
    json.dump(data, f)

In [6]:
comments = []
comments_dict = {}
for row in data: 
    json_line = json.loads(row)
    for index in range(20):
        comments.append(json_line['reviews'][index]['review'])
    comments_dict[uuid] = comments

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
comments_df = pd.Series(comments_dict, name='comments')
comments_df

945360    [abongus, not writing a reveiieiww but yes \n\...
Name: comments, dtype: object

In [14]:
class Field:
    """
    Representational json from steam
    """
    data = []

    def __init__(self):
        self._load_data()

    def _load_data(self) -> None: 
        with open('final_data_new.json', 'r', encoding="utf-8") as file:
            for line in file:
                json_line = json.loads(line,  object_hook=BlankDict)
                for index in range(len(json_line)):
                    if json_line[index]['publisher'] is not None:

                        rec = Record.record(img_url=json_line[index]['img_url'], date=json_line[index]['date'], 
                                            developer=json_line[index]['developer'], publisher=json_line[index]['publisher'], 
                                            full_desc=json_line[index]['full_desc'], 
                                            requirements=json_line[index]['requirements'], popu_tags=json_line[index]['popu_tags'], 
                                            price=json_line[index]['price'], url_info=json_line[index]['url_info'], name=json_line[index]['name'], 
                                            categories=json_line[index]['categories'])
                        self.data.append(rec)
        file.close()

class BlankDict(dict):
    def __missing__(self, key):
        return ''

class Record: 
    """
    Representation of a record
    """
    field = ['img_url', 'date', 'developer', 'publisher', 'full_desc', 'requirements', 'popu_tags', 
             'price', 'url_info', 'name', 'categories']
    record = namedtuple('Record', field)

field = Field()
field.__init__()
print(field.data[0])

Record(img_url='https://steamcdn-a.akamaihd.net/steam/apps/945360/header.jpg?t=1598556351', date='Nov 16, 2018', developer='Innersloth', publisher='Innersloth', full_desc={'sort': 'game', 'desc': "About This Game Play with 4-10 player online or via local WiFi as you attempt to prepare your spaceship for departure, but beware as one or more random players among the Crew are Impostors bent on killing everyone!Originally created as a party game, we recommend playing with friends at a LAN party or online using voice chat. Enjoy cross-platform play between Android, iOS and PC. Win by completing tasks to prepare the ship or ejecting all Impostors. React quickly to undo the Impostor's sabotages. Check the Admin map and Security cameras to keep tabs on other Crewmates. Report any dead bodies immediately to start discussion of who the suspected Impostor is. Call emergency meetings to discuss suspicious behavior. Vote to eject suspected Impostors. Kill crewmates and frame bystanders. Pretend to 

In [56]:
clean_df = pd.DataFrame(field.data, dtype='object', columns=Record.field)

In [57]:
clean_df.shape

(149642, 11)

In [58]:
clean_df.head(5)

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,"{'sort': 'game', 'desc': 'About This Game Play...",{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,{'url': 'https://store.steampowered.com/app/94...,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross..."
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,"{'sort': 'game', 'desc': 'About This Game Coun...",{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,{'url': 'https://store.steampowered.com/app/73...,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt..."
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,"{'sort': 'game', 'desc': 'About This Game Fall...",{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,{'url': 'https://store.steampowered.com/app/10...,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ..."
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,"{'sort': 'game', 'desc': 'About This Game Para...",{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,{'url': 'https://store.steampowered.com/app/11...,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St..."
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,"{'sort': 'game', 'desc': 'About This Game Dive...",{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,{'url': 'https://store.steampowered.com/app/10...,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach..."


In [59]:
clean_df.tail(5)

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories
149637,https://steamcdn-a.akamaihd.net/steam/apps/134...,"21 Jul, 2020",BioWare,,"{'sort': 'content', 'desc': 'About This Conten...",{'minimum': {'windows': {'processor': ' AMD At...,"[Massively, Multiplayer, +]",860,{'url': 'https://store.steampowered.com/app/13...,STAR WARS™: The Old Republic™ - Sith Bundles,[MMODownloadable Content]
149638,https://steamcdn-a.akamaihd.net/steam/apps/134...,Late 2020,Lychee Game Labs,,"{'sort': 'game', 'desc': 'About This Game Patc...",{'minimum': {'windows': {'processor': ' Core i...,"[Early, Access, Action, Roguelike, Roguevania,...",,,Patch Quest,"[Single-playerShared/Split Screen, Co-opFull c..."
149639,https://steamcdn-a.akamaihd.net/steam/apps/134...,"6 Nov, 2020",七彩绘色游戏,,"{'sort': 'game', 'desc': 'About This Game GALG...",{'minimum': {'linux': {'processor': ' Intel酷睿i...,"[Casual, Visual, Novel, Shooter, Nudity, Relax...",,,球球少女,"[Single-playerSteam is, learning about, this g..."
149640,https://steamcdn-a.akamaihd.net/steam/apps/134...,"20 Apr, 2021",Stegalosaurus Game Development,,"{'sort': 'game', 'desc': 'About This Game Ever...","{'minimum': {'windows': {'processor': '', 'mem...","[Dating, Romance, Dragons, Visual, Novel, Inte...",,,Furries & Scalies & Bears OH MY! 2: Return to ...,"[Single-playerSteam Achievements Steam, is lea..."
149641,https://steamcdn-a.akamaihd.net/steam/apps/597...,"12 May, 2017",IzHard,,"{'sort': 'game', 'desc': 'About This Game OVIV...",{'minimum': {'windows': {'processor': ' 1 Ghz'...,"[Indie, Adventure, Platformer, Minimalist2, Pu...",102,{'url': 'https://store.steampowered.com/app/59...,OVIVO,"[Single-playerSteam Achievements Steam, Tradin..."
